In [1]:
import os
import time
from selenium import webdriver
import pytube

In [2]:
# open browser
def open_browser():
    chrome_driver = os.environ['CHROMEDRIVER'] # path to chrome driver
    browser = webdriver.Chrome(chrome_driver)
    browser.maximize_window()
    return browser

# download the video at a given url
def download_video(url):
    youtube = pytube.YouTube(url)
    video = youtube.streams.first()
    video.download()

# scrape ijf website
def download_fight_videos():
    browser = open_browser()
    for rank in range(3,10):
        try:
            # open ranking list
            browser.get('https://www.ijf.org/wrl?category=all')
            time.sleep(2)
            # open judoka's page
            judoka = browser.find_element_by_xpath('//*[@id="wrl"]/div[2]/div[2]/div[2]/table/tbody/tr[' + str(rank) + ']/td[4]/a')
            judoka.click()
            time.sleep(2)
            # open video section
            browser.get(browser.current_url + '/videos')
            time.sleep(3)
            # get url of fight
            fight_url = browser.find_element_by_id('widget2').get_attribute('src')
            # download fight video
            download_video(fight_url)
            time.sleep(2)
        # handle error such as private videos
        except:
            print(f"error for athlete nb {rank}")
            pass
    browser.quit()

In [4]:
download_fight_videos()

error for athlete nb 5
error for athlete nb 6
